Full duplex radio를 위한 연습



In [1]:
import numpy as np
import tensorflow as tf

$s[n]$ : transmitted signal at time $n$, (real number)  
$h_p[n]$ : the channel coefficient of the $p$-th order  
$P$ : the largest order

\begin{align}
y[n]= \sum_\limits{p=1,p=odd}^{P}\sum\limits_{l=0}^{L} h_{p}[l]s[n-l]^{p}.      ~~~~~~~~~~ (1)
\end{align}

In [2]:
length_data = 100000  # length of s[n]
L = 3                 # length of delay profile [h[0], h[1], ..., h[L-1]]
var_S = 5e-1          # variance of transmitted signal
var_N = 1e-5          # variance of noise 

P_max = 5             # the largest order
P_min = 1             # the smallest order 

# traning set 
s = np.sqrt(var_S)*np.random.randn(length_data)     # the transmitted signal
y = np.zeros(length_data + L -1,dtype='float32')    # the received signal 

#########################################################
##  Training Data Set
########################################################

## Received symbols according to (1)
h = np.random.randn(P_max+1,L)
for p in [a for a in range(P_min,P_max+1) if a%2==1]:
    y = y + np.convolve(h[p],np.power(s,p))
### additive noise ###
y = y + np.sqrt(var_N)*np.random.randn(length_data + L-1)

Our objective is to estimate $\{h_p[0], h_p[1], h_p[2]\}$ for $p = 1, 3, 5$ assuming we do not know them.   
We will apply the linear regression method first to estimate $\{h_p[0], h_p[1], h_p[2]\}$. 


$y[n]$ is affected by $\{s[n], s[n-1], s[n-2], s^3[n], s^3[n-1], s^3[n-2], s^5[n], s^5[n-1], s^5[n-2]\}$.  
We can generate training set such as

$s[2], s[1], s[0], s^3[2], s^3[1], s^3[0], s^5[2], s^5[1], s^5[0], y[2]$  
$s[3], s[2], s[1], s^3[3], s^3[2], s^3[1], s^5[3], s^5[2], s^5[1], y[3]$  
$s[4], s[3], s[2], s^3[4], s^3[3], s^3[2], s^5[4], s^5[3], s^5[2], y[4]$  
$\vdots$  $\vdots$  
where the last element of each row is a target data and the rest of the elements of each row is feature data. 




In [3]:
## Training data generation 
x_data = np.zeros([length_data - L+ 1, L],dtype = 'float32')   ## feature data
y_data = np.zeros([length_data - L + 1,1],dtype = 'float32')   ## target data

for i in range(length_data - L + 1):
    y_data[i] = y[i + L -1]
    x_data[i] = s[i : i + L]

x_data = np.concatenate((x_data, np.power(x_data,3), np.power(x_data,5)),axis=1)
train_data = x_data


In [4]:
s.shape

(100000,)

We also generate test data set. 

In [5]:
###################################################    
## Test Data Set
####################################################
len_test_data = 100000  # length of s[n] for test data set 

y = np.zeros(len_test_data + L -1,dtype='float32')
s = np.sqrt(var_S)*np.random.randn(len_test_data) 

h = np.random.randn(P_max+1,L)
for p in [a for a in range(P_min,P_max+1) if a%2==1]:
    y = y + np.convolve(h[p],np.power(s,p))
y = y + np.sqrt(var_N)*np.random.randn(length_data + L-1)

x_data_test = np.zeros([length_data-L+1,L],dtype = 'float32')
y_data_test = np.zeros([length_data-L+1,1],dtype = 'float32')  ## target data

for i in range(length_data- L + 1):
    y_data_test[i] = y[i + L -1]
    x_data_test[i] = s[i : i + L]   
    
x_data_test = np.concatenate((x_data_test, np.power(x_data_test,3), np.power(x_data_test,5)),axis=1)
test_data = x_data_test

In [6]:
print(x_data.shape)
print(x_data_test.shape)

(99998, 9)
(99998, 9)


Now, with the generated train and test data, estimate $\{h_p[0], h_p[1], h_p[2]\}$ for $p = 1, 3, 5$ using linear regression method.

In [7]:
print(y_data.shape)
print(y_data_test.shape)

(99998, 1)
(99998, 1)


In [8]:
y_data

array([[ 2.803828  ],
       [-2.863088  ],
       [-2.079739  ],
       ...,
       [-0.38316968],
       [ 1.7764328 ],
       [ 0.3969923 ]], dtype=float32)

In [9]:
import sympy
import numpy

from matplotlib import pyplot
%matplotlib inline

In [10]:
# synthetic data
#x_data = numpy.linspace(-5, 5, 100)
w = 2
b = 1
numpy.random.seed(0)
z_data = w * y_data + b + numpy.random.normal(size=len(y_data))
lin_reg = 1 / (1+ numpy.exp(-z_data))

pyplot.scatter(y_data, lin_reg, alpha=0.4);

MemoryError: Unable to allocate 74.5 GiB for an array with shape (99998, 99998) and data type float64